# Low-Rank Tensor Completion (LRTC-TNN)


In [ ]:
import numpy as np
from numpy.linalg import inv as inv

def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

def mat2ten(mat, tensor_size, mode):
    index = list()
    index.append(mode)
    for i in range(tensor_size.shape[0]):
        if i != mode:
            index.append(i)
    return np.moveaxis(np.reshape(mat, list(tensor_size[index]), order = 'F'), 0, mode)

def svt_tnn(mat, alpha, rho, theta):
    """This is a Numpy dependent singular value thresholding (SVT) process."""
    u, s, v = np.linalg.svd(mat, full_matrices = 0)
    vec = s.copy()
    vec[theta :] = s[theta :] - alpha / rho
    vec[vec < 0] = 0
    return u @ np.diag(vec) @ v

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def LRTC(dense_tensor, sparse_tensor, alpha, rho, theta, 
         epsilon = 1e-4, maxiter = 100):
    """Low-Rank Tenor Completion with Truncated Nuclear Norm, LRTC-TNN."""
    
    dim = np.array(sparse_tensor.shape)
    pos_missing = np.where(sparse_tensor == 0)
    pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))
    
    X = np.zeros(np.insert(dim, 0, len(dim))) # \boldsymbol{\mathcal{X}}
    T = np.zeros(np.insert(dim, 0, len(dim))) # \boldsymbol{\mathcal{T}}
    Z = sparse_tensor.copy()
    last_tensor = sparse_tensor.copy()
    snorm = np.sqrt(np.sum(sparse_tensor ** 2))
    it = 0
    while True:
        rho = min(rho * 1.05, 1e5)
        for k in range(len(dim)):
            X[k] = mat2ten(svt_tnn(ten2mat(Z - T[k] / rho, k), alpha[k], rho, theta), dim, k)
        Z[pos_missing] = np.mean(X + T / rho, axis = 0)[pos_missing]
        T = T + rho * (X - np.broadcast_to(Z, np.insert(dim, 0, len(dim))))
        tensor_hat = np.einsum('k, kmnt -> mnt', alpha, X)
        tol = np.sqrt(np.sum((tensor_hat - last_tensor) ** 2)) / snorm
        last_tensor = tensor_hat.copy()
        it += 1
        if (it + 1) % 50 == 0:
            print('Iter: {}'.format(it + 1))
            print('MAPE: {:.6}'.format(compute_mape(dense_tensor[pos_test], tensor_hat[pos_test])))
            print('RMSE: {:.6}'.format(compute_rmse(dense_tensor[pos_test], tensor_hat[pos_test])))
            print()
        if (tol < epsilon) or (it >= maxiter):
            break

    print('Imputation MAPE: {:.6}'.format(compute_mape(dense_tensor[pos_test], tensor_hat[pos_test])))
    print('Imputation RMSE: {:.6}'.format(compute_rmse(dense_tensor[pos_test], tensor_hat[pos_test])))
    print()
    
    return tensor_hat

## On the Seattle Freeway Traffic Speed Dataset

### Random Missing Data

In [ ]:
import numpy as np
import time

for r in [0.3, 0.7, 0.9]:
    print('Missing rate = {}'.format(r))
    missing_rate = r

    ## Random missing (RM)
    dense_tensor = np.load('tensor.npz')['arr_0'].transpose(0, 2, 1)
    dim1, dim2, dim3 = dense_tensor.shape
    np.random.seed(1000)
    sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim2, dim3) + 0.5 - missing_rate)

    for theta in [5, 10, 15, 20, 25]:
        print('theta = {}'.format(theta))
        start = time.time()
        alpha = np.ones(3) / 3
        lmbda = 1e-5
        tensor_hat = LRTC(dense_tensor[:, :, : 14], sparse_tensor[:, :, : 14],
                          alpha, lmbda, theta)
        end = time.time()
        print('Running time: %d seconds'%(end - start))
        print()

In [ ]:
import numpy as np
import time

for r in [0.3]:
    print('Missing rate = {}'.format(r))
    missing_rate = r

    ## Random missing (RM)
    dense_tensor = np.load('tensor.npz')['arr_0'].transpose(0, 2, 1)
    dim1, dim2, dim3 = dense_tensor.shape
    np.random.seed(1000)
    sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim2, dim3) + 0.5 - missing_rate)

    for theta in [5]:
        print('theta = {}'.format(theta))
        start = time.time()
        alpha = np.ones(3) / 3
        lmbda = 1e-5
        tensor_hat = LRTC(dense_tensor[:, :, 14 :], sparse_tensor[:, :, 14 :],
                          alpha, lmbda, theta)
        end = time.time()
        print('Running time: %d seconds'%(end - start))
        print()

### Non-Random Missing Data

In [ ]:
import numpy as np
import time

for r in [0.3, 0.7]:
    print('Missing rate = {}'.format(r))
    missing_rate = r

    ## Non-random Missing (NM)
    dense_tensor = np.load('tensor.npz')['arr_0'].transpose(0, 2, 1)
    dim1, dim2, dim3 = dense_tensor.shape
    np.random.seed(1000)
    sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim3) + 0.5 - missing_rate)[:, None, :]

    for theta in [5, 10, 15, 20, 25]:
        print('theta = {}'.format(theta))
        start = time.time()
        alpha = np.ones(3) / 3
        lmbda = 1e-5
        tensor_hat = LRTC(dense_tensor[:, :, : 14], sparse_tensor[:, :, : 14],
                          alpha, lmbda, theta)
        end = time.time()
        print('Running time: %d seconds'%(end - start))
        print()

### Block-Out Missing

In [ ]:
import numpy as np
import time

for r in [0.3]:
    print('Missing rate = {}'.format(r))
    missing_rate = r

    ## Block-out Missing (BM)
    dense_tensor = np.load('tensor.npz')['arr_0'].transpose(0, 2, 1)
    dim1, dim2, dim3 = dense_tensor.shape
    block_window = 12
    np.random.seed(1000)
    vec = np.random.rand(int(dim2 * dim3 / block_window))
    temp = np.array([vec] * block_window)
    vec = temp.reshape([dim2 * dim3], order = 'F')
    sparse_tensor = dense_tensor * mat2ten(np.ones((dim1, dim2 * dim3)) * np.round(vec + 0.5 - missing_rate)[None, :], np.array([dim1, dim2, dim3]), 0)

    for theta in [5, 10, 15, 20, 25]:
        print('theta = {}'.format(theta))
        start = time.time()
        alpha = np.ones(3) / 3
        lmbda = 1e-5
        tensor_hat = LRTC(dense_tensor[:, :, : 14], sparse_tensor[:, :, : 14],
                          alpha, lmbda, theta)
        end = time.time()
        print('Running time: %d seconds'%(end - start))
        print()

## On the Portland Traffic Volume Dataset

### Random Missing

In [ ]:
import numpy as np
import time

for r in [0.3, 0.7, 0.9]:
    print('Missing rate = {}'.format(r))
    missing_rate = r

    # Random Missing (RM)
    dense_mat = np.load('volume.npy')
    dim1, dim2 = dense_mat.shape
    dim = np.array([dim1, 96, 31])
    dense_tensor = mat2ten(dense_mat, dim, 0)
    np.random.seed(1000)
    sparse_tensor = mat2ten(dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate), dim, 0)

    for theta in [5, 10, 15, 20, 25]:
        print('theta = {}'.format(theta))
        start = time.time()
        alpha = np.ones(3) / 3
        lmbda = 1e-5
        tensor_hat = LRTC(dense_tensor[:, :, : 14], sparse_tensor[:, :, : 14],
                          alpha, lmbda, theta)
        end = time.time()
        print('Running time: %d seconds'%(end - start))
        print()

### Non-Random Missing

In [ ]:
import numpy as np
import time

for r in [0.3, 0.7]:
    print('Missing rate = {}'.format(r))
    missing_rate = r

    # Non-random Missing (NM)
    dense_mat = np.load('volume.npy')
    dim1, dim2 = dense_mat.shape
    dim = np.array([dim1, 96, 31])
    dense_tensor = mat2ten(dense_mat, dim, 0)
    np.random.seed(1000)
    sparse_tensor = dense_tensor * np.round(np.random.rand(dim1, dim[2]) + 0.5 - missing_rate)[:, None, :]

    for theta in [5, 10, 15, 20, 25]:
        print('theta = {}'.format(theta))
        start = time.time()
        alpha = np.ones(3) / 3
        lmbda = 1e-5
        tensor_hat = LRTC(dense_tensor[:, :, : 14], sparse_tensor[:, :, : 14],
                          alpha, lmbda, theta)
        end = time.time()
        print('Running time: %d seconds'%(end - start))
        print()

### Block-Out Missing

In [ ]:
import numpy as np
import time

for r in [0.3]:
    print('Missing rate = {}'.format(r))
    missing_rate = r

    ## Block-out Missing (BM)
    dense_mat = np.load('volume.npy')
    dim1, dim2 = dense_mat.shape
    dim = np.array([dim1, 96, 31])
    dense_tensor = mat2ten(dense_mat, dim, 0)
    block_window = 4
    np.random.seed(1000)
    vec = np.random.rand(int(dim2 / block_window))
    temp = np.array([vec] * block_window)
    vec = temp.reshape([dim2], order = 'F')
    sparse_tensor = mat2ten(dense_mat * np.round(vec + 0.5 - missing_rate)[None, :], dim, 0)

    for theta in [5, 10, 15, 20, 25]:
        print('theta = {}'.format(theta))
        start = time.time()
        alpha = np.ones(3) / 3
        lmbda = 1e-5
        tensor_hat = LRTC(dense_tensor[:, :, : 14], sparse_tensor[:, :, : 14],
                          alpha, lmbda, theta)
        end = time.time()
        print('Running time: %d seconds'%(end - start))
        print()

### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>